# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# TODO: Import the necessary libs
# For example: 
import os

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import BaseMessage, UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool

import json
import chromadb
from dotenv import load_dotenv
from tavily import TavilyClient
from typing import TypedDict, List, Optional, Union, Literal
from lib.tooling import Tool, ToolCall, tool
from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Run
from pydantic import BaseModel, Field
from typing_extensions import Annotated
from lib.memory import ShortTermMemory

In [3]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [4]:
class GameFormat(BaseModel):
    """Represents a evaluation report"""
    Platform: Annotated[str, Field(description="Platform for the gamee like Game Boy, Playstation 5, Xbox 360...")]
    Name: Annotated[str, Field(description="Name of the Game")]
    YearOfRelease: Annotated[int, Field(description="Year when that game was released for that platform")]
    Description: Annotated[str, Field(description="Additional details about the game")]

class GameList(BaseModel):
    """A structured summary of a meeting"""
    ListOfGames: Annotated[List[GameFormat], Field(description="List of games in required format")]

class EvaluationReport(BaseModel):
    """Represents a evaluation report"""
    usability: Annotated[str, Field(description="whether the documents are useful to answer the question")]
    description: Annotated[str, Field(description="description about the evaluation result")]

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [5]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# collection = chroma_client.get_collection("udaplay")
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay")

@tool
def retrieve_game(query):
    """
    Semantic search: Finds most results in the vector DB
    Results: A list where each element contains -
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    args:
        query (str) : a question about game industry
    """

    
    data = collection.query(
        query_texts=query,
        n_results=3,
        include=['metadatas', 'documents', 'distances']
    )
    

    results = [
        {
            "Platform": item['Platform'],
            "Name": item["Name"],
            "YearOfRelease": item["YearOfRelease"],
            "Description": item["Description"]
        }
        for item in data['metadatas'][0]
    ]
    
    return results

In [24]:
retrieve_game("Which one was the first 3D platformer Mario game?")

[{'Platform': 'Nintendo 64',
  'Name': 'Super Mario 64',
  'YearOfRelease': 1996,
  'Description': "A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach."},
 {'Platform': 'Super Nintendo Entertainment System (SNES)',
  'Name': 'Super Mario World',
  'YearOfRelease': 1990,
  'Description': 'A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land from Bowser.'},
 {'Platform': 'Nintendo Switch',
  'Name': 'Mario Kart 8 Deluxe',
  'YearOfRelease': 2017,
  'Description': 'An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics.'}]

#### Evaluate Retrieval Tool

In [6]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result
@tool
def evaluate_retrieval(question, retrieved_docs):
    """
    Based on the user's question and on the list of retrieved documents,
    it will analyze the usability of the documents to respond to that question.
    args:
        question (str) : original question from user
        retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """

    role = "Retrieval Tool Evaluator"
    instructions = (
        "Your task is to evaluate if the documents are enough to respond the query. \n"
        "Give a detailed explanation, if it's possible to take an action on the query based on the documents"
    )
    user_message = (
        f"Analyze the user's question {question} and on the list of retrieved documents {retrieved_docs}, \n"
        "and determine the usability of the documents to respond to that question. \n"
        "The response should include : \n"
        "usabiliyt: whether the documents are useful to answer the question"
        "description: description about the evaluation result"
    )

    messages = [
        SystemMessage( content = f" You are {role}. {instructions}"),
        UserMessage( content = user_message)
    ]

    model: str = "gpt-4o-mini"
    temperature: float = 0.0

    eval_llm = LLM(
        model = model,
        temperature = temperature,
    )

    ai_message = eval_llm.invoke(messages, EvaluationReport)
    respone = json.loads(ai_message.content)


    return respone



In [8]:
question = "Which one was the first 3D platformer Mario game?"

retr_docs = retrieve_game(question)

eval_response = evaluate_retrieval(question, retr_docs)
print(type(eval_response))
print(eval_response)

<class 'dict'>
{'usability': 'Yes', 'description': "The documents retrieved include information about various Mario games, specifically highlighting 'Super Mario 64' as a groundbreaking 3D platformer released in 1996. This directly answers the user's question about which was the first 3D platformer Mario game. The other documents mention 'Super Mario World' and 'Mario Kart 8 Deluxe', but they are not relevant to the query regarding 3D platformers. Therefore, the document about 'Super Mario 64' is sufficient to respond to the user's question."}


#### Game Web Search Tool

In [7]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 


@tool
def game_web_search(question: str, search_depth: Literal["basic", "advanced"] = "advanced"):
    """
    Semantic search: Finds most results in the vector DB
    args:
    - question: a question about game industry.
    """
    client = TavilyClient(api_key=TAVILY_API_KEY)

    search_depth = search_depth.strip().lower()

    #perform the search
    search_result = client.search(
        query = question,
        search_depth = search_depth,
        include_answer = True,
        include_raw_content=False,
        include_images=False
    )

    model: str = "gpt-4o-mini"
    temperature: float = 0.0

    search_llm = LLM(
        model = model,
        temperature = temperature,
    )

    user_message = f"Extract text from {search_result} and format the message by Platform, Name, YearOfRelease and Description"
    messages = [
        UserMessage( content = user_message)
    ]

 

    ai_message = search_llm.invoke(messages, GameList)
    results = json.loads(ai_message.content)


    return results['ListOfGames']



In [10]:
tavily_question = "Which one was the first 3D platformer Mario game?"
tav_result = game_web_search(tavily_question)

print(tav_result)

#parsed = json.loads(tav_result)

#parsed['ListOfGames']


[{'Platform': 'Nintendo 64', 'Name': 'Super Mario 64', 'YearOfRelease': 1996, 'Description': 'Super Mario 64 was the first 3D platformer in the Mario series, introducing many gameplay mechanics that became staples in later titles. It is widely acclaimed as one of the greatest games in video game history, revolutionizing the genre with its fully realized 3D gameplay.'}, {'Platform': 'PC', 'Name': 'Alpha Waves', 'YearOfRelease': 1990, 'Description': 'Alpha Waves is often cited as the first 3D platformer, featuring fully 3D levels where players jump to reach goals. However, it is not considered a true 3D platformer by many due to its lack of analog movement and gameplay that still felt constrained to 2D.'}]


### Agent

In [8]:
# Define the AgentState TypedDict
# Include fields for user_query, instructions, messages, and current_tool_calls

class AgentState(TypedDict):
    user_query: str  # The current user query being processed
    instructions: str  # System instructions for the agent
    messages: List[dict]  # List of conversation messages
    current_tool_calls: Optional[List[ToolCall]]  # Current pending tool calls
    session_id: str  # Session identifier for memory management


In [9]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed

class UdaplayAgent:
    def __init__(self, 
                 model_name: str,
                 instructions: str, 
                 tools: List[Tool] = None,
                 temperature: float = 0.7):
        """
        Initialize a MemoryAgent instance
        
        Args:
            model_name: Name/identifier of the LLM model to use
            instructions: System instructions for the agent
            tools: Optional list of tools available to the agent
            temperature: Temperature parameter for LLM (default: 0.7)
        """
        self.instructions = instructions
        self.tools = tools if tools else []
        self.model_name = model_name
        self.temperature = temperature
        
        # Initialize memory and state machine
        self.memory = ShortTermMemory()
        self.workflow = self._create_state_machine()

    def _prepare_messages_step(self, state: AgentState) -> AgentState:
        """Step logic: Prepare messages for LLM consumption"""
        messages = state.get("messages", [])
        
        # If no messages exist, start with system message
        if not messages:
            messages = [SystemMessage(content=state["instructions"])]
            
        # Add the new user message
        messages.append(UserMessage(content=state["user_query"]))
        
        return {
            "messages": messages,
            "session_id": state["session_id"]
        }

    def _llm_step(self, state: AgentState) -> AgentState:
        """Step logic: Process the current state through the LLM"""
        # Initialize LLM
        llm = LLM(
            model=self.model_name,
            temperature=self.temperature,
            tools=self.tools
        )

        response = llm.invoke(state["messages"])
        tool_calls = response.tool_calls if response.tool_calls else None

        # Create AI message with content and tool calls
        ai_message = AIMessage(content=response.content, tool_calls=tool_calls)
        
        return {
            "messages": state["messages"] + [ai_message],
            "current_tool_calls": tool_calls,
            "session_id": state["session_id"]
        }

    def _tool_step(self, state: AgentState) -> AgentState:
        """Step logic: Execute any pending tool calls"""
        tool_calls = state["current_tool_calls"] or []
        tool_messages = []
        
        for call in tool_calls:
            # Access tool call data correctly
            function_name = call.function.name
            function_args = json.loads(call.function.arguments)
            tool_call_id = call.id
            # Find the matching tool
            tool = next((t for t in self.tools if t.name == function_name), None)
            if tool:
                result = tool(**function_args)
                tool_message = ToolMessage(
                    content=json.dumps(result), 
                    tool_call_id=tool_call_id, 
                    name=function_name, 
                )
                tool_messages.append(tool_message)
        
        # Clear tool calls and add results to messages
        return {
            "messages": state["messages"] + tool_messages,
            "current_tool_calls": None,
            "session_id": state["session_id"]
        }

    def _create_state_machine(self) -> StateMachine[AgentState]:
        """Create the internal state machine for the agent"""
        machine = StateMachine[AgentState](AgentState)
        
        # Create steps
        entry = EntryPoint[AgentState]()
        message_prep = Step[AgentState]("message_prep", self._prepare_messages_step)
        llm_processor = Step[AgentState]("llm_processor", self._llm_step)
        tool_executor = Step[AgentState]("tool_executor", self._tool_step)
        termination = Termination[AgentState]()
        
        machine.add_steps([entry, message_prep, llm_processor, tool_executor, termination])
        
        # Add transitions
        machine.connect(entry, message_prep)
        machine.connect(message_prep, llm_processor)
        
        # Transition based on whether there are tool calls
        def check_tool_calls(state: AgentState) -> Union[Step[AgentState], str]:
            """Transition logic: Check if there are tool calls"""
            if state.get("current_tool_calls"):
                return tool_executor
            return termination
        
        machine.connect(llm_processor, [tool_executor, termination], check_tool_calls)
        machine.connect(tool_executor, llm_processor)  # Go back to llm after tool execution
        
        return machine

    def invoke(self, query: str, session_id: Optional[str] = None) -> Run:
        """
        Run the agent on a query
        
        Args:
            query: The user's query to process
            session_id: Optional session identifier (uses "default" if None)
            
        Returns:
            The final run object after processing
        """
        session_id = session_id or "default"

        # Create session if it doesn't exist
        self.memory.create_session(session_id)

        # Get previous messages from last run if available
        previous_messages = []
        last_run: Run = self.memory.get_last_object(session_id)
        if last_run:
            last_state = last_run.get_final_state()
            if last_state:
                previous_messages = last_state["messages"]

        initial_state: AgentState = {
            "user_query": query,
            "instructions": self.instructions,
            "messages": previous_messages,
            "current_tool_calls": None,
            "session_id": session_id,
        }

        run_object = self.workflow.run(initial_state)
        
        # Store the complete run object in memory
        self.memory.add(run_object, session_id)
        
        return run_object

    def get_session_runs(self, session_id: Optional[str] = None) -> List[Run]:
        """Get all Run objects for a session
        
        Args:
            session_id: Optional session ID (uses "default" if None)
            
        Returns:
            List of Run objects in the session
        """
        return self.memory.get_all_objects(session_id)

    def reset_session(self, session_id: Optional[str] = None):
        """Reset memory for a specific session
        
        Args:
            session_id: Optional session to reset (uses "default" if None)
        """
        self.memory.reset(session_id)

In [10]:
tools = [retrieve_game, evaluate_retrieval, game_web_search]
udaplay_insructions = (
    "You are a helpful gaming analyst agent good with awering questions on games\n"
    "You wil a logical step by step sequenial approach to get answer to the question\n"
    "- You will first try to get the answer from retrieval of game details avalable locally\n"
    "- You will then evaluate the retrieved data to see if it can be used to answer he question\n"
    "- If it is not possible to annswer, you will do a web search to get the answer\n "
    "When you think it is over, return the finl answer\n"
    "Never try to respond directly if the question need a tool\n"
    f"The tools that you have for action are Tools : {tools}. \n"
    "Present the output in the following format\n"
    "- Platform: like Game Boy, Playstation 5, Xbox 360...)\n"
    "- Name: Name of the Game\n"
    "- YearOfRelease: Year when that game was released for that platform\n"
    "- Description: Additional details about the game"
)


udaplay_agent = UdaplayAgent(
    model_name = "gpt-4o-mini",
    tools = tools,
    instructions = udaplay_insructions
)

In [22]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

query1 = "When Pokémon Gold and Silver was released?"
query2 = "Which one was the first 3D platformer Mario game?"
query31 = "Was Mortal Kombat X realeased for Playstation 5?"
query32 = "What upgrade was released subsequent to release for Paystation 5?"


In [12]:
udaplay_run_object_1 = udaplay_agent.invoke(query = query1, session_id = "Pokemon")
messages1 = udaplay_run_object_1.get_final_state()["messages"]

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [16]:
udaplay_run_object_2 = udaplay_agent.invoke(query = query2, session_id = "Mario")
messages2 = udaplay_run_object_2.get_final_state()["messages"]

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [19]:
udaplay_run_object_31 = udaplay_agent.invoke(query = query31, session_id = "Mortal")
messages31 = udaplay_run_object_31.get_final_state()["messages"]

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [23]:
udaplay_run_object_32 = udaplay_agent.invoke(query = query32, session_id = "Mortal")
messages32 = udaplay_run_object_32.get_final_state()["messages"]

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


## Reporting the output

In [13]:
import json
from pprint import pprint

def pretty_print_messages(question, messages):
    print("\n" + "=" * 80)
    print("🧠 AGENT EXECUTION TRACE")
    print("=" * 80)
    print(f"Question : {question}\n")
    print("Execution Trace : \n")


    for i, msg in enumerate(messages, start=1):
        role = getattr(msg, "role", "unknown")
        content = getattr(msg, "content", None)
        tool_calls = getattr(msg, "tool_calls", None)

        print(f"\n🔹 Step {i} | Role: {role.upper()}")

        if content:
            # Try to pretty-print JSON content
            try:
                parsed = json.loads(content)
                print("📄 Content (JSON):")
                pprint(parsed, indent=2)
            except Exception:
                print("📄 Content:")
                print(content)

        if tool_calls:
            print("🛠 Tool Calls:")
            for tc in tool_calls:
                print(f"  • Tool: {tc.function.name}")
                try:
                    args = json.loads(tc.function.arguments)
                    print("    Arguments:")
                    pprint(args, indent=4)
                except Exception:
                    print(f"    Raw arguments: {tc.function.arguments}")

    print("\n" + "=" * 80)

def print_final_answer(question, messages):
    print("\n" + "=" * 80)
    print("FINAL OUTPUT")
    print("=" * 80)
    print(f"Question : {question}\n")
    print("Final Output : \n")

    for msg in reversed(messages):
        if msg.role == "assistant" and msg.content:
            print(msg.content)
            return

    print("\n" + "=" * 80)


In [14]:
print_final_answer(query1, messages1)


FINAL OUTPUT
Question : When Pokémon Gold and Silver was released?

Final Output : 

- Platform: Game Boy Color
- Name: Pokémon Gold and Silver
- YearOfRelease: 1999
- Description: Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.


In [15]:
pretty_print_messages(query1, messages1)


🧠 AGENT EXECUTION TRACE
Question : When Pokémon Gold and Silver was released?

Execution Trace : 


🔹 Step 1 | Role: SYSTEM
📄 Content:
You are a helpful gaming analyst agent good with awering questions on games
You wil a logical step by step sequenial approach to get answer to the question
- You will first try to get the answer from retrieval of game details avalable locally
- You will then evaluate the retrieved data to see if it can be used to answer he question
- If it is not possible to annswer, you will do a web search to get the answer
 When you think it is over, return the finl answer
Never try to respond directly if the question need a tool
The tools that you have for action are Tools : [<Tool name=retrieve_game params=['query']>, <Tool name=evaluate_retrieval params=['question', 'retrieved_docs']>, <Tool name=game_web_search params=['question', 'search_depth']>]. 
Present the output in the following format
- Platform: like Game Boy, Playstation 5, Xbox 360...)
- Name: Name of

In [17]:
print_final_answer(query2, messages2)


FINAL OUTPUT
Question : Which one was the first 3D platformer Mario game?

Final Output : 

The first 3D platformer Mario game is:

- Platform: Nintendo 64
- Name: Super Mario 64
- YearOfRelease: 1996
- Description: A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.


In [18]:
pretty_print_messages(query2, messages2)


🧠 AGENT EXECUTION TRACE
Question : Which one was the first 3D platformer Mario game?

Execution Trace : 


🔹 Step 1 | Role: SYSTEM
📄 Content:
You are a helpful gaming analyst agent good with awering questions on games
You wil a logical step by step sequenial approach to get answer to the question
- You will first try to get the answer from retrieval of game details avalable locally
- You will then evaluate the retrieved data to see if it can be used to answer he question
- If it is not possible to annswer, you will do a web search to get the answer
 When you think it is over, return the finl answer
Never try to respond directly if the question need a tool
The tools that you have for action are Tools : [<Tool name=retrieve_game params=['query']>, <Tool name=evaluate_retrieval params=['question', 'retrieved_docs']>, <Tool name=game_web_search params=['question', 'search_depth']>]. 
Present the output in the following format
- Platform: like Game Boy, Playstation 5, Xbox 360...)
- Name: 

In [20]:
print_final_answer(query31, messages31)


FINAL OUTPUT
Question : Was Mortal Kombat X realeased for Playstation 5?

Final Output : 

Mortal Kombat X was not released for the PlayStation 5. Here are the details:

- **Platform**: PlayStation 4
- **Name**: Mortal Kombat X
- **YearOfRelease**: 2015
- **Description**: Mortal Kombat X is a fighting game developed by NetherRealm Studios and published by Warner Bros. It features iconic characters and brutal combat mechanics.

Additionally, an upgraded version called Mortal Kombat XL was released in 2016 for PlayStation 4, but it also did not have a specific release for PlayStation 5.


In [21]:
pretty_print_messages(query31, messages31)


🧠 AGENT EXECUTION TRACE
Question : Was Mortal Kombat X realeased for Playstation 5?

Execution Trace : 


🔹 Step 1 | Role: SYSTEM
📄 Content:
You are a helpful gaming analyst agent good with awering questions on games
You wil a logical step by step sequenial approach to get answer to the question
- You will first try to get the answer from retrieval of game details avalable locally
- You will then evaluate the retrieved data to see if it can be used to answer he question
- If it is not possible to annswer, you will do a web search to get the answer
 When you think it is over, return the finl answer
Never try to respond directly if the question need a tool
The tools that you have for action are Tools : [<Tool name=retrieve_game params=['query']>, <Tool name=evaluate_retrieval params=['question', 'retrieved_docs']>, <Tool name=game_web_search params=['question', 'search_depth']>]. 
Present the output in the following format
- Platform: like Game Boy, Playstation 5, Xbox 360...)
- Name: N

In [24]:
print_final_answer(query32, messages32)


FINAL OUTPUT
Question : What upgrade was released subsequent to release for Paystation 5?

Final Output : 

The following upgrades and releases were noted for the PlayStation 5:

1. **Platform**: PlayStation 5
   - **Name**: PlayStation 5 Pro
   - **YearOfRelease**: 2026
   - **Description**: The PlayStation 5 Pro upgrade is set for release in 2026, featuring unspecified improvements.

2. **Platform**: PlayStation 5
   - **Name**: Various PS4 Titles
   - **YearOfRelease**: 2025
   - **Description**: Many PS4 games can be upgraded into PS5 versions that have major enhancements.

3. **Platform**: PlayStation 5
   - **Name**: Stellaris: Console Edition
   - **YearOfRelease**: 2025
   - **Description**: Recent PS5 upgrade scheduled for November 6th, 2025.

4. **Platform**: PlayStation 5
   - **Name**: Turok 2: Seeds of Evil
   - **YearOfRelease**: 2025
   - **Description**: Recent PS5 upgrade scheduled for October 31st, 2025.

5. **Platform**: PlayStation 5
   - **Name**: Virtua Fighter 5

In [25]:
pretty_print_messages(query32, messages32)


🧠 AGENT EXECUTION TRACE
Question : What upgrade was released subsequent to release for Paystation 5?

Execution Trace : 


🔹 Step 1 | Role: SYSTEM
📄 Content:
You are a helpful gaming analyst agent good with awering questions on games
You wil a logical step by step sequenial approach to get answer to the question
- You will first try to get the answer from retrieval of game details avalable locally
- You will then evaluate the retrieved data to see if it can be used to answer he question
- If it is not possible to annswer, you will do a web search to get the answer
 When you think it is over, return the finl answer
Never try to respond directly if the question need a tool
The tools that you have for action are Tools : [<Tool name=retrieve_game params=['query']>, <Tool name=evaluate_retrieval params=['question', 'retrieved_docs']>, <Tool name=game_web_search params=['question', 'search_depth']>]. 
Present the output in the following format
- Platform: like Game Boy, Playstation 5, Xbox 

## Check Session History

In [27]:

def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")
        
print("Mortal session runs:")
runs = udaplay_agent.get_session_runs("Mortal")
for i, run_object in enumerate(runs, 1):
    print(f"\n# Run {i}", run_object.metadata)
    print("Messages:")
    print_messages(run_object.get_final_state()["messages"])

Mortal session runs:

# Run 1 {'run_id': '1e099cd7-be4c-4d8d-afd7-8a3622d4ee8a', 'start_timestamp': '2025-12-15 13:19:52.282863', 'end_timestamp': '2025-12-15 13:20:03.420719', 'snapshot_counts': 7}
Messages:
 -> (role = system, content = You are a helpful gaming analyst agent good with awering questions on games
You wil a logical step by step sequenial approach to get answer to the question
- You will first try to get the answer from retrieval of game details avalable locally
- You will then evaluate the retrieved data to see if it can be used to answer he question
- If it is not possible to annswer, you will do a web search to get the answer
 When you think it is over, return the finl answer
Never try to respond directly if the question need a tool
The tools that you have for action are Tools : [<Tool name=retrieve_game params=['query']>, <Tool name=evaluate_retrieval params=['question', 'retrieved_docs']>, <Tool name=game_web_search params=['question', 'search_depth']>]. 
Present th

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes